## Project Name
> Auxilary tools and solution for pre/post
processing of audio and video based on ffmpeg
inspired by [ffmpeg]('https://pypi.org/project/python-ffmpeg-video-streaming/)

FFmpeg is an extremely powerful and versatile command-line tool for converting audio and video files. It is free and available for Windows, Mac and Linux machines. Whether you want to join two video files, extract the audio component from a video file, convert your video into an animated GIF, FFmpeg can do it all and even more.

# 1. Installing/updates of libraries

In [1]:

# from datetime import datetime
!pip install python-ffmpeg-video-streaming
!apt install ffmpeg
!pip install ffmpy

from IPython.display import HTML
from IPython.display import Image


from base64 import b64encode

import ffmpeg_streaming
from ffmpeg_streaming import Formats
import ffmpy
import os
import datetime as dt
import shutil, json



Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
  Created wheel for ffmpy: filename=ffmpy-0.3.0-cp36-none-any.whl size=4710 sha256=d01643c8a6fa30adba29eaa0f831204d734b40ee1673ab35ce85f52264aed646
  Stored in directory: /root/.cache/pip/wheels/cc/ac/c4/bef572cb7e52bfca170046f567e64858632daf77e0f34e5a74
Successfully built ffmpy


# 2. Mounting google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Standard snipet to prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [4]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


## Check GPU

*   Google Colab can provide you with one of Tesla graphics cards: K80, T4, P4 or P100
*   Here you can check the model of GPU before using some advanced features

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Creating/checking if exist temporary folder '/content/drive/MyDrive/ff_temp/' for pre/post processing

# Useful functions

In [5]:
def folder_check(path):
    try_num = 1
    oripath = path[:-1] if path.endswith('/') else path
    if(not os.path.exists(path)):
      os.mkdir(path)
      print(path, '   created')
    else:    
      while os.path.exists(path):
          print("Delete content of  existing folder " + path + "?(Y/N)")
          decision = input()
          if decision == "Y":
              shutil.rmtree(path, ignore_errors=True)
              os.mkdir(path)

              break
          else:
            path = oripath + "_%d/"%try_num
            try_num += 1
            if(not os.path.exists(path)):
              print("Create new folder " + path + "?(Y/N)")
              decision = input()
              if decision == "Y":
                os.mkdir(path)
                print(path, 'was created')
                break
      return path

In [6]:
# generating file name with timestamp info for later comparison
def gen_file_with_timestamp(name='name'):
    timestamp = dt.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".mp4"
    return file_name

In [ ]:
filename = gen_file_with_timestamp(name='name')
filename

'nameSunday14February202102-13-26-07.mp4'

In [ ]:
def save_m3u8(path,  time_chunk):
  print('chunks will be saved in {} directory to stop press stop button on the left'.format(path) )

  os.system(f"ffmpeg -re -i https://tagesschau-lh.akamaihd.net/i/tagesschau_1@119231/master.m3u8 -vcodec libx264 -vb 500000 -g 60 -vprofile main -acodec aac -ab 128000 -ar 48000 -ac 2 -vbsf h264_mp4toannexb -b 1000k -minrate 1000k -maxrate 1000k -strict experimental -f stream_segment -segment_format mpegts -segment_time {time_chunk} -segment_atclocktime 1 -reset_timestamps 1 -strftime 1 {path}%H%M%S.mp4")

# 3. Playing video file in colab window

In [ ]:
video_file='/content/drive/MyDrive/ff_temp/204800recod.mp4'
mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.



> Preparing temporary folder for video processing



In [ ]:
path = folder_check('/content/drive/MyDrive/ff_temp/')

Delete content of  existing folder /content/drive/MyDrive/ff_temp/?(Y/N)
N
Delete content of  existing folder /content/drive/MyDrive/ff_temp_1/?(Y/N)
Y


In [ ]:
os.system(f"ffmpeg -i {input_video} -vcodec libx264 {output_video}")

0



# 3a. Decoding once again mp4  If it gives error there is need to decode pice of mp4 again like below



In [ ]:
input_video='/content/drive/MyDrive/ff_temp/205101.mp4'
output_video='/content/drive/MyDrive/ff_temp/205101recod.mp4'

In [ ]:
def recode_mp4(input_video):

  output_video= input_video[:-4]+'recode'+'.mp4'
  os.system(f"ffmpeg -i {input_video} -vcodec libx264 {output_video}")
  print('recoded video is  saved to {} '.format(output_video))
  return output_video

In [ ]:
#@title Decode once again with ffmpeg { display-mode: "form" }

input_video = "/content/drive/MyDrive/tesser/video/m2.mp4" #@param [""] {allow-input: true}
print('video {}  is recoded please wait...  '.format(input_video))
output_video= recode_mp4(input_video)




As a source of video for later checking  saved m3u8 video stream will be used

In [ ]:
time_chunk = 210 # in seconds
save_m3u8(path,  time_chunk)

chunks will be saved in /content/drive/MyDrive/ff_temp_2/ directory to stop press stop button on the left


In [ ]:
path = '/content/drive/MyDrive/ff_temp/'

In [ ]:
def recode_mp4_from_folder(path):
  to_recode = [path+s  for s in os.listdir(path) if s.endswith('.mp4')]

  #creating list of tuples source and result name
  to_recode_out=[]
  for i in to_recode:
    to_recode_out.append((i, i[:-4]+'recode'+i[-4:]))
    
  #recoding saved chunks
  for  j in to_recode_out:
    print(j[0], ' --> vcodec libx264 --> ',j[1] )
    os.system(f"ffmpeg -i {j[0]} -vcodec libx264 {j[1]}")
    os.remove(j[0])
    print(j[0], ' --> was removed after recoding ')

In [ ]:
recode_mp4_from_folder(path)

/content/drive/MyDrive/ff_temp/200001.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/200001recode.mp4
/content/drive/MyDrive/ff_temp/200001.mp4  --> was removed after recoding 
/content/drive/MyDrive/ff_temp/201500.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/201500recode.mp4
/content/drive/MyDrive/ff_temp/201500.mp4  --> was removed after recoding 
/content/drive/MyDrive/ff_temp/204800.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/204800recode.mp4
/content/drive/MyDrive/ff_temp/204800.mp4  --> was removed after recoding 
/content/drive/MyDrive/ff_temp/205101.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/205101recode.mp4
/content/drive/MyDrive/ff_temp/205101.mp4  --> was removed after recoding 


#4. Cutting with reencoding 

Nice and short

In [ ]:
# os.system(f"ffmpeg -i {input_video} -ss {start_end[0]} -to {start_end[1]} -c:v libx264 -crf 30 {output_video}")

0

In [15]:
#start_end is defined as tuple start_end=('0:01', '0:30')

def trim_mp4(input_video, start_end):

  output_video= input_video[:-4]+'_trimed_'+start_end[0]+'_to_'+start_end[1]+input_video[-4:]
  print('trimed video will be  saved to {} ... please wait'.format(output_video))
  os.system(f"ffmpeg -i {input_video} -ss {start_end[0]} -to {start_end[1]} -c:v libx264 -crf 30 {output_video}")
  return output_video

In [ ]:
# start_end=('0:01', '0:30')
# input_video='/content/drive/MyDrive/tesser/video/m2.mp4'
# output_video=trim_mp4(input_video, start_end)

trimed video will be  saved to /content/drive/MyDrive/tesser/video/m2_trimed_01_to_30.mp4 ...


In [ ]:
# input_video= '/content/drive/MyDrive/tesser/video/m2.mp4'
# duration=mp4_info(input_video)
# start_end=['0:01']
# end_='0:'+str(duration)
# start_end.append(end_)
# to_play=trim_mp4(input_video, start_end)

trimed video will be  saved to /content/drive/MyDrive/tesser/video/m2_trimed_01_to_38.mp4 ... please wait


In [20]:
#@title Cut 1s from start and one at the end and reencode( to avoid problem with playing in colab) { display-mode: "form" }

input_video = "/content/drive/MyDrive/video/result2012_2.mp4" #@param [""] {allow-input: true}
duration=mp4_info(input_video)



duration=mp4_info(input_video)
start_end=['0:01']
end_='0:'+str(duration)
start_end.append(end_)

output_video=trim_mp4(input_video, start_end)



trimed video will be  saved to /content/drive/MyDrive/tesser/video/m2_trimed_0:01_to_0:38.mp4 ... please wait


In [18]:
#@title Triming and reencoding any time within duration '3:36', '3:50' { display-mode: "form" }

input_video = "/content/temp/p3.mp4" #@param [""] {allow-input: true}
trim_start= '3:36'#@param [""] {allow-input: true}
trim_end= '3:50'#@param [""] {allow-input: true}
duration=mp4_info(input_video)


start_end=[]
start_end.append(trim_start)
start_end.append(trim_end)

# print(start_end) 
output_video=trim_mp4(input_video, start_end)



['3:36', '3:50']
trimed video will be  saved to /content/temp/p3_trimed_3:36_to_3:50.mp4 ... please wait


In [22]:
#@title Play output.mp4 of last executed code { display-mode: "form" }

video_file = "/content/temp/p3_trimed_3:36_to_3:50.mp4" #@param [""] {allow-input: true}


mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""video
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# 5. Extract the audio from video

The -vn switch extracts the audio portion from a video and we are using the -ab switch to save the audio as a 256kbps MP3 audio file.

In [ ]:
def extract_mp3(input_video, sampling_rate):

  output_audio= input_video[:-4]+'.mp3'
  print('extracted audio will be  saved to {} ...'.format(output_audio))
  os.system(f"ffmpeg -i {input_video} -vn -ab {sampling_rate} {output_audio}")
  return output_audio

In [ ]:
sampling_rate=256
input_video='/content/drive/MyDrive/ff_temp/200001recode_trimed_01_to_43.mp4'
audio_file= extract_mp3(input_video, sampling_rate)

extracted audio will be  saved to /content/drive/MyDrive/ff_temp/200001recode_trimed_01_to_43.mp3 ...


In [ ]:
mp4 = open(audio_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# 6. Convert a video into an animated GIF

FFmpeg is an excellent tool for converting videos into animated GIFs and the quality isn’t bad either. Use the scale filter to specify the width of the GIF, the -t parameter specific the duration while -r specifies the frame rate (fps).

In [ ]:
def convert_video_to_gif(input_video, scale, t, framerate):

  output_gif= input_video[:-4]+'.gif'
  print('video will be  saved to {} ...'.format(output_gif))
  os.system(f"ffmpeg -i {input_video} -vf scale={scale}:-1 -t {t} -r {framerate} {output_gif}")
  return output_gif

In [ ]:
scale=500
t=10
framerate=10
input_video='/content/drive/MyDrive/antek/20210210_123015.mp4'
output_gif=convert_video_to_gif(input_video, scale, t, framerate)

video will be  saved to /content/drive/MyDrive/antek/20210210_123015.gif ...


In [ ]:
Image(open(output_gif,'rb').read())

In [ ]:
#@title converting to gif with form { display-mode: "form" }
file_to_process = "/content/drive/MyDrive/antek/20210210_123015.mp4" #@param [""] {allow-input: true}
scale =  300#@param {type:"integer"}
framerate = 10 #@param {type:"integer"}
t = 14 #@param {type:"integer"}
output_gif=convert_video_to_gif(file_to_process, scale, t, framerate)

video will be  saved to /content/drive/MyDrive/antek/20210210_123015.gif ...


In [ ]:
Image(open(output_gif,'rb').read())

# 7.Info about the video file in json format

In [ ]:
# input_video="/content/drive/MyDrive/ff_temp/200001recode_trimed_01_to_43_faded.mp4"
# json_file= input_video[:-4]+'_json.json'
# os.system(f"ffprobe -v quiet -print_format json -show_format -show_streams {input_video} > {json_file}")


0

In [8]:
def mp4_info(input_video):
  json_file= input_video[:-4]+'_params.json'
  os.system(f"ffprobe -v quiet -print_format json -show_format -show_streams {input_video} > {json_file}")
  json1 = json.loads(open(json_file).read())
  value = json1['streams'][0]['duration']
  duration=int(float(value))
  return duration

In [ ]:
input_video_s=['/content/drive/MyDrive/ff_temp/200001recode_trimed_01_to_43_faded.mp4','/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55.mp4']

In [ ]:
for i in input_video_s:
  duration = mp4_info(i)
  print(i, '----duration-->' ,duration)

/content/drive/MyDrive/ff_temp/200001recode_trimed_01_to_43_faded.mp4 ----duration--> 42
/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55.mp4 ----duration--> 25


# 8. Fade in  and fade out (video)

In [ ]:
def fade_in_out_mp4_simple(input_video1, fade_in_out):

  output_video= output_video= input_video[:-4]+'_faded.mp4'
  print('Fade_in fade_out video will be  saved to {} ...'.format(output_video))
  os.system(f"ffmpeg -i {input_video1} -vf {fade_in_out} -c:a copy {output_video}")
  return output_video

In [ ]:
def fade_in_out_mp4(input_video1, fade_param):

  json_file= input_video[:-4]+'_param.json'
  os.system(f"ffprobe -v quiet -print_format json -show_format -show_streams {input_video} > {json_file}")

  json1 = json.loads(open(json_file).read())
  value = json1['streams'][0]['duration']
  duration=int(float(value))
  
  #generating string for fade in and fade out for ffmpeg

  
  fade_param[2]=duration-fade_param[3]
  fade_in_out='fade=t=in:st='+str(fade_param[0])+':d='+str(fade_param[1])+',fade=t=out:st='+str(fade_param[2])+':d='+(str(fade_param[3]))
  print('Duration = 3rd param was changed to  :',fade_in_out) #preparing input string for fading funcion

  output_video= output_video= input_video[:-4]+'_v_faded.mp4'
  print('Fade_in fade_out video will be  saved to {} ...'.format(output_video))
  os.system(f"ffmpeg -i {input_video1} -vf {fade_in_out} -c:a copy {output_video}")
  return output_video

In [ ]:
input_video="/content/drive/MyDrive/ff_temp/200001recode.mp4"
fade_param=[0,10,10,5] #start_fade_in, duration_fade_in start_fade_out, duration_fade_out
video_file= fade_in_out_mp4(input_video, fade_param)

fade=t=in:st=0:d=10,fade=t=out:st=115:d=5
Fade_in fade_out video will be  saved to /content/drive/MyDrive/ff_temp/200001recode_faded.mp4 ...


In [ ]:
mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""video
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.

# Fade in and fade out with sliders

In [ ]:
#@title Fade_in and out duration times( clip duration is checked automatically) { display-mode: "form" }
# Data = "data_dst" #@param ["data_src", "data_dst"]
file_to_process = "/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55.mp4" #@param [""] {allow-input: true}
duration_fade_in = 2 #@param {type:"slider", min:0, max:5, step:1}
duration_fade_out = 3 #@param {type:"slider", min:0, max:5, step:1}
# input_video="/content/drive/MyDrive/ff_temp/200001recode.mp4"
fade_param=[0,10,10,5] #start_fade_in, duration_fade_in start_fade_out, duration_fade_out
fade_param[1]= duration_fade_in
fade_param[3] = duration_fade_out
print(fade_param)
# fade_param
input_video = file_to_process # this important as when you take directly file to process it take last
video_file= fade_in_out_mp4(input_video, fade_param)

[0, 2, 10, 3]
Duration = 3rd param was changed to  : fade=t=in:st=0:d=2,fade=t=out:st=22:d=3
Fade_in fade_out video will be  saved to /content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55_v_faded.mp4 ...


In [ ]:
#@title Play output.mp4 of last executed code { display-mode: "form" }

video_file = "/content/drive/MyDrive/tesser/video/m2_trimed_01_to_38.mp4" #@param [""] {allow-input: true}


mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""video
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.

# 9. Extract frames from video to folder

In [ ]:
def png_from_mp4(input_video, every_th_sec): # .5 means two times more as duration in second, 0.04 so every 25th frame
  png_folder=input_video[:-4]+'_PNG_'
  path= folder_check(png_folder)
  print('png files will be saved in : ', png_folder )
  os.system(f"ffmpeg -i {input_video} -r {every_th_sec} {path}/frames_%04d.png")

In [ ]:
every_th_sec=25
input_video='/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55.mp4'
png_from_mp4(input_video, every_th_sec)

Delete content of  existing folder /content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55_PNG_?(Y/N)
Y
png files will be saved in :  /content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55_PNG_


# 10.Merge a folder of PNG images to a mp4

-pix_fmt yuv420p is very important otherwise merged video will not be played

In [ ]:
def mp4_from_png(input_folder, framerate): 
  mp4_file=input_folder[:-4]+'_conv_to_mp4.mp4'
  # path= folder_check(png_folder)
  print('png files will be onverted to mp4  : ', mp4_file )
  input_folder_files=input_folder+'/*.png'
  input_folder_files='\"{}\"'.format(input_folder_files) # way to put brackets into formatted string those extra https://stackoverflow.com/questions/12338818/how-can-i-get-double-quotes-into-a-string-literal#12338826

  print('files frto be converted : ',input_folder_files )
  # !ffmpeg -framerate {framerate} -pattern_type glob  -i "{input_folder_files}" -pix_fmt yuv420p {mp4_file} #that works
  os.system(f"ffmpeg -framerate {framerate} -pattern_type glob  -i {input_folder_files} -pix_fmt yuv420p {mp4_file}") #that works
  

  return mp4_file

In [ ]:
#@title Merge a folder of PNG images to a mp4 { display-mode: "form" }
# Data = "data_dst" #@param ["data_src", "data_dst"]
folder_with_png_to_process = "/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55_PNG_" #@param [""] {allow-input: true}
frame_rate = 25 #@param {type:"slider", min:1, max:30, step:1}

framerate=frame_rate
input_folder=folder_with_png_to_process
video_out= mp4_from_png(input_folder, framerate)

png files will be onverted to mp4  :  /content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55__conv_to_mp4.mp4
"/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55_PNG_/*.png"


In [ ]:
#@title Play output.mp4 of last executed code( just copy/paste) { display-mode: "form" }

video_file = "/content/drive/MyDrive/ff_temp/201500recode_trimed_30_to_55__conv_to_mp4.mp4" #@param [""] {allow-input: true}


mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""video
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# 11. Crop video with particular pixel coordinates

The syntax for cropping a video is crop=width:height:x:y. With this, here is a command line that crops a 200×200 portion of a video.



```
ffmpeg -i input.mp4 -filter_complex "[0:v]crop=200:200:300:100[cropped]" -map "[cropped]" output.mp4
```



This command results in a 200x200 cropped portion of the video situated 300 pixels from the left edge and 100 pixels from the top edge.

If the parameters 300:100 is not provided, the resulting portion will be taken from the center of the video. It’s that easy! 

In [ ]:
input_video='/content/drive/MyDrive/ff_temp/pdb/antibody.mp4'
mp4_info(input_video)

31

In [ ]:
import json

In [ ]:
def mp4_w_h(json_file): # returnig width and hight of video
  
  json1 = json.loads(open(json_file).read())
  h= json1['streams'][0]['height']
  w= json1['streams'][0]['width']
  return w, h

In [ ]:
def check_if_video_crop_possible(crop_param, h, w):
  if w>(crop_param[0]+crop_param[2]) and h>(crop_param[1]+crop_param[3]):
    print( 'Video crop will be done..')
    return 10
  else: print( 'Please change crop parameters !')

In [ ]:
def mp4_crop_px(input_video, crop_param): 
  mp4_file=input_video[:-4]+'_crop.mp4'
  print(mp4_file)
  fil_complex='\"[0:v]crop={}:{}:{}:{}[cropped]\"'.format(crop_param[0],crop_param[1], crop_param[2],crop_param[3] )
  cropp= '\"[cropped]\"'  
  os.system(f"ffmpeg -i {input_video} -filter_complex {fil_complex} -map {cropp} {mp4_file}") 

  return mp4_file

In [ ]:
crop_param=[940,520,50,155] #output wideo witht, output video height, pixels from the left edge, pixels from the top edge

In [ ]:
#@title Video crop with particular pixel coordinates { display-mode: "form" }
file_to_process = "/content/drive/MyDrive/ff_temp/pdb/antibody.mp4" #@param [""] {allow-input: true}
px_from_the_left = 50 #@param {type:"integer"}
px_from_the_top = 155 #@param {type:"integer"}

input_video = file_to_process # this important as when you take directly file to process it take last
crop_param[2]= px_from_the_left
crop_param[3] = px_from_the_top
print(crop_param, ' output wideo width, output video height, pixels from the left edge, pixels from the top edge')
print('Input  -->   {} ---> width {} height {}'.format(input_video, w, h))
w, h= mp4_w_h(json_file)
poss=check_if_video_crop_possible(crop_param, h, w)
if poss==10:
  print('croping video...')
  mp4_crop_px(input_video, crop_param)


[940, 520, 50, 155]  output wideo width, output video height, pixels from the left edge, pixels from the top edge
Input  -->   /content/drive/MyDrive/ff_temp/pdb/antibody.mp4 ---> width 1280 height 720
Video crop will be done..
croping video...
/content/drive/MyDrive/ff_temp/pdb/antibody_crop.mp4


In [ ]:
#@title Play output.mp4 of last executed code( just copy/paste) { display-mode: "form" }

video_file = "/content/drive/MyDrive/ff_temp/pdb/antibody_crop.mp4" #@param [""] {allow-input: true}


mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""video
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
scale=500
t=32
framerate=10
input_video='/content/drive/MyDrive/ff_temp/pdb/antibody_crop.mp4'
output_gif=convert_video_to_gif(input_video, scale, t, framerate)

video will be  saved to /content/drive/MyDrive/ff_temp/pdb/antibody_crop.gif ...


In [ ]:
Image(open(output_gif,'rb').read())

Output hidden; open in https://colab.research.google.com to view.

# 11. Slow down audio and video

In [23]:
# !cp /content/temp/p3_trimed_3:36_to_3:50.mp4 /content/drive/MyDrive/ff_temp/piano

In [ ]:
# !ffmpeg -i /content/drive/MyDrive/ff_temp/piano/p3.mp4 -filter_complex setpts=4*PTS;atempo=0.5,atempo=0.5  /content/drive/MyDrive/ff_temp/piano/out.mp4

In [47]:
!ffmpeg -i /content/drive/MyDrive/ff_temp/piano/p3.mp4  -filter_complex "[0:v]setpts=PTS*4[v];[0:a]atempo=0.5,atempo=0.5[a]" -map "[v]" -map "[a]" /content/drive/MyDrive/ff_temp/piano/0_25x_output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [48]:
#@title Play output.mp4 of last executed code( just copy/paste) { display-mode: "form" }

video_file = "/content/drive/MyDrive/ff_temp/piano/0_25x_output.mp4" #@param [""] {allow-input: true}


mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""video
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.